In [2]:
import org.apache.spark.sql.SparkSession
val sc = SparkSession.builder().getOrCreate()
sc.version

sc = org.apache.spark.sql.SparkSession@7ba4c2f9


2.4.3

In [3]:
import org.apache.spark.sql.functions.{to_date, col}
val df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("./all/*.csv").coalesce(5)
val dfWithDate = df.withColumn("date", to_date(col("InvoiceDate"), "MM/d/yyyy H:mm"))

df = [InvoiceNo: string, StockCode: string ... 6 more fields]
dfWithDate = [InvoiceNo: string, StockCode: string ... 7 more fields]


[InvoiceNo: string, StockCode: string ... 7 more fields]

In [4]:
val dfNoNull = dfWithDate.drop()

dfNoNull = [InvoiceNo: string, StockCode: string ... 7 more fields]


[InvoiceNo: string, StockCode: string ... 7 more fields]

### 7.4.1 롤업
다양한 컬럼을 그룹화 키로 설정된 조합뿐 아니라 데이터셋에서 볼 수 있는 실제 조합을 모두 살펴볼 수 있음. <br>
null 값을 가진 로우에서 전체 날짜의 합계를 확인할 수 있습니다. 롤업된 두 개의 칼럼값이 모두 null인 로우는 두 컬럼에 속한 레코드의 전체 합계를 나타냄.

In [ ]:
import org.apache.spark.sql.functions.sum
val rolledUpDf = dfNoNull.rollup("Date", "Country").agg(sum("Quantity"))
    .selectExpr("Date", "Country", "`sum(Quantity)` as total_quantity").orderBy("Date")
rolledUpDf.show()

### 7.4.2 큐브
롤업을 고차원적으로 사용할 수 있게 함. 큐브는 요소들을 계층적으로 다루는 대신 모든 차원에 대해 동일한 작업을 수행함. <br>
- 전체 날짜와 모든 국가에 대한 합계
- 모든 국가의 날짜별 합계
- 날짜별 국가별 합계
- 전체 날짜의 국가별 합계

In [ ]:
val rolledUpDf = dfNoNull.cube("Date", "Country").agg(sum("Quantity"))
    .selectExpr("Date", "Country", "`sum(Quantity)` as total_quantity").orderBy("Date")
rolledUpDf.show()

### 7.4.3 그룹화 메타데이터
큐브와 롤업을 사용하다 보면 집계 수준에 따라 쉽게 필터링하기 위해 집계 수준을 조회하는 경우가 발생함. <br>
grouping_id를 사용해 집계 수준을 명시하는 컬럼을 제공함
- 0: customerId, stockCode별 조합에 따라 총 수량 제공
- 1: 구매한 물품에 관계 없이 customerId를 기반으로 총 수량 제공
- 2: 개별 재고 코드의 모든 집계 결과를 나타냄
- 3: 가장 높은 계층의 집계 결과를 나타냄. customerId나 stockCode에 관계 없이 총 수량을 제공

In [2]:
import org.apache.spark.sql.functions.{grouping_id, sum, expr, col}
val rolledUpDf = dfNoNull.cube("customerId", "stockCode").agg(grouping_id(), sum("Quantity"))
    .orderBy(col("grouping_id()").desc)
rolledUpDf.show()

Name: Compile Error
Message: <console>:26: error: not found: value dfNoNull
       val rolledUpDf = dfNoNull.cube("customerId", "stockCode").agg(grouping_id(), sum("Quantity"))
                        ^

StackTrace: 

### 7.4.4 피벗
로우를 컬럼으로 변환할 수 있음. 현재 데이터셋에서는 Country 컬럼이 있음.  <br>
피벗을 사용해 국가별로 집계 함수를 적용할 수 있으며 쿼리를 사용해 쉽게 결과를 확인할 수 있음.|

In [3]:
val pivoted = dfWithDate.groupBy("date").pivot("Country").sum()

Name: Compile Error
Message: <console>:25: error: not found: value dfWithDate
       val pivoted = dfWithDate.groupBy("date").pivot("Country").sum()
                     ^

StackTrace: 

### 7.5 사용자 정의 집계 함수
직접 제작한 함수, 비즈니스 규칙에 기반을 둔 자체 집계 함수를 정의하는 방법 <br>
UDAF를 사용해 입력 데이터 그룹에 직접 개발한 연산을 수행할 수 있다. <br> 
중간 결과를 단일 AggregationBuffer에 저장해 관리한다.

In [ ]:
import org.apache.spark.sql.expression.MutableAggregationBuffer
import org.apache.spark.sql.expression.UserDefinedAggregateFunction
import org.apache.spark.sql.Row
import org.apache.spark.sql.types._

class BoolAnd extends UserDefinedAggregateFunction {
    def inputSchema: org.apache.spark.sql.types.StructType =
        StructType(StructField("value", BooleanType) :: Nil)
    def bufferSchema: StrucType = StrucType(StrucField("result", BooleanType) :: Nil)
    def dataType: DataType = BooleanType
    def deterministric: Boolean = true
    def initialize(buffer: MutableAggregationBuffer): Unit = {
        buffer(0) = true
    }
    
    def update(buffer: MutableAggregationBuffer, input: Row): Unit = {
        buffer(0) = buffer.getAs[Boolean](0) && input.getAs[Boolean](0)
    }
    def merge(buffer1: MutableAggregationBuffer, buffer2: Row): Unit = {
        buffer1(0) = buffer1.getAs[Boolean](0) && buffer2.getAs[Boolean](0)
    }
    def evaluate(buffer: Row): Any = {
        buffer(0)
    }
}

val ba = new BoolAnd
spark.udf.register("boolean", ba)

import org.apache.spark.sql.functions._
spark.range(1)
    .selectExpr("explode(array(TRUE, TRUE, TRUE)) as t")
    .selectExpr("explode(array(TRUE, FALSE, TRUE)) as f", "t")
    .select(ba(col("t")), expr("booland(f)")).show()